# 🤖 多机器人开发调试笔记本

本笔记本用于开发和调试 Telegram 多机器人系统：
- 机器人配置加载和验证
- 人设（personality）测试
- 技能（skills）匹配测试
- Agent 路由测试
- 数据库操作测试

In [ ]:
# 环境设置
import sys
import os
from pathlib import Path

# 添加项目根目录到路径
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

# 设置必要的环境变量（测试用）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

print(f"项目根目录: {project_root}")
print("✅ 环境设置完成")

## 1. 加载所有机器人配置

In [ ]:
from src.bot.config_loader import BotConfigLoader, get_config_loader

# 创建配置加载器
loader = BotConfigLoader(bots_dir=str(project_root / "bots"))

# 列出所有可用的 Bot
available_bots = loader.list_bots()

print(f"🤖 发现 {len(available_bots)} 个可用的机器人:")
print("=" * 50)
for bot_id in available_bots:
    config = loader.load_config(bot_id)
    if config:
        print(f"\n📌 {bot_id}")
        print(f"   名称: {config.name}")
        print(f"   描述: {config.description}")
        print(f"   类型: {config.bot_type}")

## 2. 详细查看单个机器人配置

In [ ]:
# 选择要查看的机器人
BOT_ID = "teacher_bot"  # 可以修改为其他机器人ID

config = loader.load_config(BOT_ID)

if config:
    print(f"📋 {BOT_ID} 详细配置")
    print("=" * 60)
    
    # 基础信息
    print(f"\n📝 基础信息:")
    print(f"  名称: {config.name}")
    print(f"  描述: {config.description}")
    print(f"  用户名: @{config.username}")
    print(f"  类型: {config.bot_type}")
    print(f"  语言: {config.language}")
    print(f"  是否公开: {config.is_public}")
    
    # AI 配置
    print(f"\n🤖 AI 配置:")
    print(f"  Provider: {config.ai.provider}")
    print(f"  模型: {config.ai.model}")
    print(f"  温度: {config.ai.temperature}")
    print(f"  最大Token: {config.ai.max_tokens}")
else:
    print(f"❌ 未找到 {BOT_ID} 配置")

In [ ]:
# 查看系统提示词
if config:
    system_prompt = config.get_system_prompt()
    
    print(f"🎯 系统提示词 (System Prompt):")
    print("-" * 60)
    print(system_prompt)
    print("-" * 60)
    print(f"\n📊 提示词长度: {len(system_prompt)} 字符")

## 3. 测试人设配置 (Personality)

In [ ]:
import yaml

# 直接加载 YAML 查看完整人设配置
bot_config_path = project_root / "bots" / BOT_ID / "config.yaml"

with open(bot_config_path, 'r', encoding='utf-8') as f:
    raw_config = yaml.safe_load(f)

personality = raw_config.get('personality', {})

print(f"🎭 {BOT_ID} 人设配置:")
print("=" * 60)

# 角色描述
if 'character' in personality:
    print(f"\n📖 角色描述:")
    print(personality['character'])

# 性格特点
if 'traits' in personality:
    print(f"\n✨ 性格特点:")
    for trait in personality['traits']:
        print(f"  • {trait}")

# 语言风格
if 'speaking_style' in personality:
    style = personality['speaking_style']
    print(f"\n🗣️ 语言风格:")
    print(f"  语气: {style.get('tone', 'N/A')}")
    print(f"  正式程度: {style.get('formality', 'N/A')}")
    print(f"  使用Emoji: {style.get('use_emoji', 'N/A')}")

## 4. 测试技能配置 (Skills)

In [ ]:
skills = raw_config.get('skills', {})

print(f"💪 {BOT_ID} 技能配置:")
print("=" * 60)

# 主要技能
primary_skills = skills.get('primary', [])
print(f"\n🔥 主要技能 ({len(primary_skills)} 项):")
for skill in primary_skills:
    print(f"\n  📌 {skill['name']}")
    print(f"     描述: {skill['description']}")
    print(f"     熟练度: {skill.get('proficiency', 'N/A')}")
    if 'keywords' in skill:
        print(f"     关键词: {', '.join(skill['keywords'][:5])}...")

# 次要技能
secondary_skills = skills.get('secondary', [])
print(f"\n🌟 次要技能 ({len(secondary_skills)} 项):")
for skill in secondary_skills:
    print(f"  • {skill['name']}: {skill['description']}")

## 5. 技能匹配测试

In [ ]:
def test_skill_matching(message: str, skills_config: dict) -> dict:
    """
    测试消息与技能的匹配度
    
    Args:
        message: 用户消息
        skills_config: 技能配置
        
    Returns:
        dict: 匹配结果，包含每个技能的匹配分数
    """
    results = {}
    message_lower = message.lower()
    
    # 测试主要技能
    for skill in skills_config.get('primary', []):
        skill_name = skill['name']
        keywords = skill.get('keywords', [])
        
        # 计算关键词匹配数
        match_count = sum(1 for kw in keywords if kw.lower() in message_lower)
        match_score = match_count / max(len(keywords), 1)
        
        results[skill_name] = {
            'match_count': match_count,
            'match_score': round(match_score, 2),
            'matched_keywords': [kw for kw in keywords if kw.lower() in message_lower]
        }
    
    return results

# 测试消息
test_messages = [
    "这道数学题怎么解？",
    "我想学英语，有什么好的学习方法吗？",
    "考试好紧张怎么办？",
    "物理的牛顿第三定律是什么？"
]

print(f"🎯 技能匹配测试 ({BOT_ID}):")
print("=" * 60)

for msg in test_messages:
    print(f"\n📝 消息: \"{msg}\"")
    results = test_skill_matching(msg, skills)
    
    # 找到最佳匹配
    best_match = max(results.items(), key=lambda x: x[1]['match_score'])
    
    print(f"   最佳匹配: {best_match[0]} (分数: {best_match[1]['match_score']})")
    if best_match[1]['matched_keywords']:
        print(f"   匹配关键词: {', '.join(best_match[1]['matched_keywords'])}")

## 6. Agent 系统测试

In [ ]:
from src.agents import Router, RouterConfig, Message, ChatContext, AgentLoader

# 加载所有可用的 Agents
agent_loader = AgentLoader(agents_dir=str(project_root / "agents"))
agents = agent_loader.load_agents()

print(f"📦 已加载 {len(agents)} 个 Agent:")
for agent in agents:
    print(f"  • {agent.name}: {agent.description[:50]}...")

In [ ]:
# 创建路由器
router_config = RouterConfig(
    min_confidence=0.3,
    max_agents=2,
    fallback_enabled=True
)
router = Router(agents, router_config)

# 测试消息路由
test_user_messages = [
    "今天心情不好，感觉很焦虑",
    "Python 代码报错了怎么办？",
    "能帮我写一段代码吗？",
    "最近工作压力好大"
]

print("🔀 Agent 路由测试:")
print("=" * 60)

for msg_text in test_user_messages:
    # 创建消息和上下文
    message = Message(
        content=msg_text,
        user_id="test_user_123",
        chat_id="test_chat_456"
    )
    context = ChatContext(chat_id="test_chat_456")
    
    print(f"\n📝 消息: \"{msg_text}\"")
    
    # 获取每个 Agent 的置信度
    for agent in agents:
        confidence = agent.can_handle(message, context)
        if confidence > 0:
            print(f"   {agent.name}: {confidence:.2f}")

## 7. 数据库测试

In [ ]:
# 注意：此部分需要数据库连接
# 如果没有配置数据库，可以跳过

try:
    from src.database import get_db_session
    from src.models.database import Bot, User, Channel, ChannelBotMapping
    
    db = get_db_session()
    
    print("📊 数据库状态:")
    print("=" * 60)
    print(f"  用户数: {db.query(User).count()}")
    print(f"  机器人数: {db.query(Bot).count()}")
    print(f"  频道数: {db.query(Channel).count()}")
    print(f"  绑定数: {db.query(ChannelBotMapping).count()}")
    
    # 列出所有机器人
    bots = db.query(Bot).all()
    if bots:
        print(f"\n🤖 已注册的机器人:")
        for bot in bots:
            print(f"  [{bot.id}] @{bot.bot_username} - {bot.bot_name}")
            print(f"      AI: {bot.ai_provider}/{bot.ai_model}")
    
    db.close()
    
except Exception as e:
    print(f"⚠️ 数据库连接失败: {e}")
    print("   请确保已正确配置数据库连接")

## 8. 创建新机器人模板

In [ ]:
# 机器人配置模板生成器

def generate_bot_template(bot_id: str, bot_name: str, description: str, 
                          bot_type: str = "assistant", 
                          traits: list = None,
                          primary_skills: list = None) -> str:
    """
    生成机器人配置模板
    
    Args:
        bot_id: 机器人ID（用于文件夹名）
        bot_name: 机器人显示名称
        description: 机器人描述
        bot_type: 机器人类型 (companion/assistant/expert/service)
        traits: 性格特点列表
        primary_skills: 主要技能列表
        
    Returns:
        str: YAML 配置内容
    """
    traits = traits or ["专业", "友好", "耐心"]
    primary_skills = primary_skills or [
        {"name": "通用问答", "description": "回答各类问题", "keywords": ["问题", "帮助"]}
    ]
    
    template = f'''# {bot_name} 配置文件
# {description}

bot:
  name: "{bot_name}"
  description: "{description}"
  username: "{bot_id}"
  type: "{bot_type}"
  language: "zh"
  is_public: true

personality:
  character: |
    你是{bot_name}，一个{description}的AI助手。
    请在这里添加详细的角色描述...
  
  traits:
'''
    
    for trait in traits:
        template += f'    - "{trait}"\n'
    
    template += '''
  speaking_style:
    tone: "友好专业"
    formality: "半正式"
    use_emoji: true

skills:
  primary:
'''
    
    for skill in primary_skills:
        template += f'''    - name: "{skill['name']}"
      description: "{skill['description']}"
      proficiency: "expert"
      keywords:
'''
        for kw in skill.get('keywords', []):
            template += f'        - "{kw}"\n'
    
    template += '''
ai:
  provider: "openai"
  model: "gpt-4o"
  temperature: 0.7
  max_tokens: 1500
  context_window: 4096

prompt:
  custom: |
    请在这里添加详细的系统提示词...

features:
  enabled:
    - "conversation_memory"
  disabled:
    - "code_execution"

routing:
  mode: "auto"
  private_chat_auto_reply: true
  group_chat_mention_required: true

messages:
  welcome: |
    👋 你好！我是{bot_name}。
    有什么可以帮你的吗？

metadata:
  version: "1.0.0"
  author: "Your Name"
'''
    
    return template

# 生成示例
example_template = generate_bot_template(
    bot_id="my_custom_bot",
    bot_name="MyCustomBot",
    description="我的自定义机器人",
    bot_type="assistant",
    traits=["热情", "专业", "幽默"],
    primary_skills=[
        {"name": "客服支持", "description": "回答产品相关问题", "keywords": ["帮助", "问题", "如何"]}
    ]
)

print("📝 生成的配置模板:")
print("=" * 60)
print(example_template)

## 9. 快速创建新机器人

In [ ]:
# 创建新机器人的快捷函数

def create_new_bot(bot_id: str, config_content: str, 
                   bots_dir: Path = None) -> bool:
    """
    创建新的机器人目录和配置文件
    
    Args:
        bot_id: 机器人ID
        config_content: YAML配置内容
        bots_dir: bots目录路径
        
    Returns:
        bool: 是否创建成功
    """
    if bots_dir is None:
        bots_dir = project_root / "bots"
    
    bot_dir = bots_dir / bot_id
    
    # 检查是否已存在
    if bot_dir.exists():
        print(f"⚠️ 机器人 {bot_id} 已存在")
        return False
    
    try:
        # 创建目录
        bot_dir.mkdir(parents=True, exist_ok=True)
        
        # 创建 __init__.py
        init_content = f'''"""\n{bot_id} - 自定义机器人\n"""\nfrom pathlib import Path\n\nBOT_DIR = Path(__file__).parent\nCONFIG_FILE = BOT_DIR / "config.yaml"\nBOT_ID = "{bot_id}"\n'''
        
        (bot_dir / "__init__.py").write_text(init_content)
        (bot_dir / "config.yaml").write_text(config_content)
        
        print(f"✅ 机器人 {bot_id} 创建成功!")
        print(f"   目录: {bot_dir}")
        return True
        
    except Exception as e:
        print(f"❌ 创建失败: {e}")
        return False

# 使用示例（取消注释以执行）：
# create_new_bot("my_custom_bot", example_template)

print("💡 使用 create_new_bot() 函数可以快速创建新机器人")
print("   示例: create_new_bot('my_bot', template_content)")

## 10. 调试小结

### 开发新机器人的步骤：

1. **创建目录**: 在 `bots/` 下创建新的机器人文件夹
2. **配置人设**: 在 `config.yaml` 中定义 personality 部分
3. **定义技能**: 在 `config.yaml` 中定义 skills 部分
4. **编写提示词**: 在 prompt.custom 中编写详细的系统提示词
5. **测试匹配**: 使用本笔记本测试技能匹配和路由
6. **注册数据库**: 使用 db_manager.py 将机器人注册到数据库

### 常用命令：

```bash
# 查看数据库状态
python scripts/db_manager.py status

# 创建新机器人
python scripts/db_manager.py bot

# 绑定机器人到频道
python scripts/db_manager.py bind
```